In [1]:
import pandas as pd
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np

from notears import linear, nonlinear, utils

In [2]:
def var_sort_lin(X_norm, d, sorting):
    X_varsort = X.copy()
    X_varsort[:, sorting] *= np.linspace(1, d, d)
    return X_varsort

In [3]:
def varsortability(X, W, tol=1e-9):
    """ Takes n x d data and a d x d adjaceny matrix,
    where the i,j-th entry corresponds to the edge weight for i->j,
    and returns a value indicating how well the variance order
    reflects the causal order. """
    E = W != 0
    Ek = E.copy()
    var = np.var(X, axis=0, keepdims=True)

    n_paths = 0
    n_correctly_ordered_paths = 0

    for _ in range(E.shape[0] - 1):
        n_paths += Ek.sum()
        n_correctly_ordered_paths += (Ek * var / var.T > 1 + tol).sum()
        n_correctly_ordered_paths += 1/2*(
            (Ek * var / var.T <= 1 + tol) *
            (Ek * var / var.T >  1 - tol)).sum()
        Ek = Ek.dot(E)

    return n_correctly_ordered_paths / n_paths

In [4]:
d = 10
n = 100
s0 = 10
graph_type = "ER" 

B_true = utils.simulate_dag(d, s0, graph_type)
W = utils.simulate_parameter(B_true)
X = utils.simulate_linear_sem(B_true, n, "gauss")

In [5]:
X.std(axis=0)

array([2.5406479 , 5.19532474, 5.19056424, 1.06724286, 2.82649845,
       0.90646389, 2.45922192, 1.39976952, 0.98445567, 1.10665039])

In [6]:
g = ig.Graph.Adjacency(B_true, loops=False)
g.vs["label"] = list(range(d))

sorting = g.topological_sorting()

print(sorting)

[3, 5, 8, 9, 7, 6, 0, 4, 1, 2]


In [7]:
# ORIGINAL
print("VS-original", varsortability(X, W))

# NORMALIZE
X = (X  - X.mean(axis=0)) / X.std(axis=0)
print("VS-normalised", varsortability(X, W))


# CONTROL VARSORT
X = var_sort_lin(X, d, sorting)
print("VS-controlled-lin", varsortability(X, W))

VS-original 1.0
VS-normalised 0.5
VS-controlled-lin 1.0


In [ ]:
X.std(axis=0)

In [ ]:
X_.std(axis=0)

In [ ]:
vars = np.logspace(1, d+1, d, base=0.5)
vars = np.full(vars.shape, vars.max()) - vars
vars /= (vars[-1] / (d+1))

plt.plot(np.arange(0, d, 1), vars)


In [ ]:
np.full(vars.shape, 1)

In [ ]:
varsortability(X, W)

In [ ]:
varsortability(X_